In [2]:
import numpy as np
import pandas as pd

# Idioms

## 满足某个条件下进行赋值

In [34]:
df = pd.DataFrame(
    {"AAA": [4,5,6,7], "BBB": [10,20,30,40], "CCC": [100,200,300,400], "DDD": [1,2,3,4]}
)
df

,AAA,BBB,CCC,DDD
0,4,10,100,1
1,5,20,200,2
2,6,30,300,3
3,7,40,400,4


首先需要明确df.loc的参数, 可以对多列进行条件判断, 或者赋值.

In [35]:
df.loc[(df.AAA >= 5) & (df.BBB <= 30), ["CCC", "DDD"]] = -1
df

,AAA,BBB,CCC,DDD
0,4,10,100,1
1,5,20,-1,-1
2,6,30,-1,-1
3,7,40,400,4


新建一列并根据条件赋值, 可以用np.where函数. 

In [20]:
df["logic"] = np.where(df.AAA > 5, "high", "low")
df

,AAA,BBB,CCC,logic
0,4,10,100,low
1,5,20,200,low
2,6,-1,-1,high
3,7,-1,-1,high


在R语言里，满足某个条件进行赋值采用
`df %>% filter(AAA > 5) %>% mutate(logic = "high")`
或者运用ifelse函数
`df %>% mutate()`

## 拆分split
为R语言的filter, 利用多个条件进行筛选, 具体见上节. 此处展示不同案例.

按照与某一数值的距离进行排序, 使用`argsort()`, 该函数将元素从小到大排列, 提取其对应的index, 并返回一个Series. 

注意Series是没有sort属性的. np.sort()返回值一个的列表.  

`df.loc[[0, 1, 2, 3]]`返回index按照列表顺序选择的subset. 同样的, 逗号之后是columns按照列表顺序选择的subset, 比如`df.iloc[:, [0, 3, 3, 2]]`.

In [22]:
df = pd.DataFrame(
    {"AAA": [4,5,6,7], "BBB": [10,20,30,40], "CCC": [100,20,30,-40], "DDD": [1,2,3,4]}
)
df
aValue = 43
df.loc[(df.CCC - aValue).abs().argsort()]

,AAA,BBB,CCC,DDD
2,6,30,30,3
1,5,20,20,2
0,4,10,100,1
3,7,40,-40,4


用二元操作符`&, |`或者`functools.reduce()`来构建复合的拆分条件. 

In [25]:
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)

Crit1 = df.AAA <= 5.5
Crit2 = df.BBB == 10.0
Crit3 = df.CCC > -40.0

AllCrit = Crit1 & Crit2 & Crit3

df[AllCrit]

,AAA,BBB,CCC
0,4,10,100


`functools.reduce()` 其作用是对参数序列中元素进行累积，返回值是一个数值. 它的第一个参数接收且只接收带两个参数的函数, 第二个参数是一个列表, 功能是将列表中前两个元素传入第一个参数中, 函数返回的值再和第三个元素传入第一个参数中, 最后得到一个结果. 
比如 `reduce(lambda x, y: x+y, [1, 2, 3, 4, 5])` 计算 `((((1+2)+3)+4)+5)`. 

In [26]:
import functools

Crit_list = [Crit1, Crit2, Crit3]
AllCrit = functools.reduce(lambda x, y: x & y, Crit_list)
df[AllCrit]

,AAA,BBB,CCC
0,4,10,100


## 选择Selection
为R语言的filter. 可以用数值判断`> < =`, 行\列的位置`iloc[]`或者标签`loc[]`进行筛选, 同时支持切片`loc["foo":"bar"]`. 
此处展示其他功能案例. 反向选择`~` 

In [31]:
df = pd.DataFrame(
    {"AAA": [4, 5, 6, 7], "BBB": [10, 20, 30, 40], "CCC": [100, 50, -30, -50]}
)
df
#df[(df.AAA <= 6) & (df.index.isin([0,2,4]))]
df[~((df.AAA <= 6) & (df.index.isin([0,2,4])))]

,AAA,BBB,CCC
1,5,20,50
3,7,40,-50


## 新建一列
关键函数解析. `df.columns` 返回数据框的列标签, 类似的`df.index`返回数据框的行标签. `[str(x) + "_cat" for x in source_cols]` 是一个列表推导式. `categories.get`字典的`.get()`方法按照key返回value. 

`df.applymap()`函数用于对数据框中的每一个元素执行相同的函数操作. 类似的`df.apply()`函数用于对数据框中的一列或者一行进行操作.

In [51]:
df = pd.DataFrame({"AAA": [1, 2, 1, 3], "BBB": [1, 1, 2, 2], "CCC": [2, 1, 3, 1]})
df
source_cols = df.columns
new_cols = [str(x) + "_cat" for x in source_cols]
categories = {1: "Alpha", 2: "Beta", 3: "Charlie"}
df[new_cols] = df[source_cols].applymap(categories.get)
df

,AAA,BBB,CCC,AAA_cat,BBB_cat,CCC_cat
0,1,1,2,Alpha,Alpha,Beta
1,2,1,1,Beta,Alpha,Alpha
2,1,2,3,Alpha,Beta,Charlie
3,3,2,1,Charlie,Beta,Alpha


对一列进行分组, 另一列保留最小值. 两种实现方法. 
方法一, `df.idxmin()`返回行或者列上最小值第一次出现的索引值. `.loc[]`根据索引值筛选.

In [63]:
df = pd.DataFrame(
    {"AAA": [1, 1, 1, 2, 2, 2, 3, 3], "BBB": [2, 1, 3, 4, 5, 1, 2, 3]}
)
df
df.loc[df.groupby("AAA")["BBB"].idxmin()]

,AAA,BBB
1,1,1
5,2,1
6,3,2


方法二, 用sort, 比较难懂. 

In [73]:
df.sort_values(by="BBB").groupby("AAA", as_index=False).first()

,AAA,BBB
0,1,1
1,2,1
2,3,2
